<a href="https://colab.research.google.com/github/Akshay-Kumar-Arya/Identify_the_sentiments/blob/master/models/bert_nnlm_lreg_svm_mlp_evaluation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using nnlm+bert vectors

In [34]:
import sklearn
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.metrics import f1_score

pd.set_option('display.max_colwidth', 200)

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
# Data path
training_data_path = "/content/gdrive/My Drive/Identify_the_sentiments/train.csv"
test_data_path =  "/content/gdrive/My Drive/Identify_the_sentiments/test.csv"

save_path = "/content/gdrive/My Drive/Identify_the_sentiments/"

In [7]:
# reading data from csv
train_data = pd.read_csv(training_data_path)
test_data = pd.read_csv(test_data_path)

print (train_data.shape, test_data.shape)

(7920, 3) (1953, 2)


In [8]:
# import nnlm embeddings for tweets
train_file = open(save_path + "nnlm_train.pickle", "rb")
pool_embs_train_nnlm = pickle.load(train_file)

test_file = open(save_path + "nnlm_test.pickle", "rb")
pool_embs_test_nnlm = pickle.load(test_file)

In [9]:
# import bert embeddings for tweets
train_file = open(save_path + "bert_train.pickle", "rb")
pool_embs_train_bert, _ = pickle.load(train_file)

test_file = open(save_path + "bert_test.pickle", "rb")
pool_embs_test_bert, _ = pickle.load(test_file)

In [10]:
train_final = np.concatenate((pool_embs_train_bert, pool_embs_train_nnlm),axis=1)
test_final = np.concatenate((pool_embs_test_bert, pool_embs_test_nnlm),axis=1)

In [11]:
# splitting training and validation data
xtrain, xvalid, ytrain, yvalid = train_test_split(train_final, 
                                                  train_data['label'],  
                                                  random_state=42, 
                                                  test_size=0.2)
print (xtrain.shape, xvalid.shape, ytrain.shape, yvalid.shape)

(6336, 896) (1584, 896) (6336,) (1584,)


## logistic regression

In [12]:
# logistic regression model
lreg = LogisticRegression(max_iter = 10000, warm_start= True, verbose=True)
lreg.fit(xtrain, ytrain)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.8s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=True,
                   warm_start=True)

In [13]:
# logistic regression model prediction on validation dataset
preds_valid = lreg.predict(xvalid)
print (f1_score(yvalid, preds_valid))

0.789598108747045


In [ ]:
# prediction on test dataset
preds_test = lreg.predict(test_final)


# prepare submission dataframe
sub = pd.DataFrame({'id':test_data['id'], 'label':preds_test})

# write predictions to a CSV file
sub.to_csv(save_path + "sub_lreg_bert+nnlm.csv", index=False)
# Your score for this submission is : 0.877712596054773.

## mlp

In [43]:
mlp = MLPClassifier(activation='logistic', solver='adam', alpha=1e-5,
                    hidden_layer_sizes=(200, 50), random_state=1,
                    learning_rate_init=0.01,max_iter=100, warm_start = True, learning_rate = 'invscaling', verbose= True)

mlp.fit(xtrain, ytrain)

Iteration 1, loss = 0.57576572
Iteration 2, loss = 0.54830609
Iteration 3, loss = 0.41414542
Iteration 4, loss = 0.29781562
Iteration 5, loss = 0.28458242
Iteration 6, loss = 0.27815549
Iteration 7, loss = 0.26672405
Iteration 8, loss = 0.25722559
Iteration 9, loss = 0.24634811
Iteration 10, loss = 0.25301496
Iteration 11, loss = 0.24299192
Iteration 12, loss = 0.24197727
Iteration 13, loss = 0.24787174
Iteration 14, loss = 0.23982459
Iteration 15, loss = 0.23929418
Iteration 16, loss = 0.23465909
Iteration 17, loss = 0.25078176
Iteration 18, loss = 0.25062782
Iteration 19, loss = 0.23328224
Iteration 20, loss = 0.23383266
Iteration 21, loss = 0.22604578
Iteration 22, loss = 0.23726248
Iteration 23, loss = 0.23170843
Iteration 24, loss = 0.24119061
Iteration 25, loss = 0.22666072
Iteration 26, loss = 0.22569310
Iteration 27, loss = 0.22724206
Iteration 28, loss = 0.23017333
Iteration 29, loss = 0.21950086
Iteration 30, loss = 0.23744276
Iteration 31, loss = 0.21619309
Iteration 32, los

MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(200, 50), learning_rate='invscaling',
              learning_rate_init=0.01, max_fun=15000, max_iter=100,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=True)

In [44]:
# mlp regression model prediction on validation dataset
preds_valid = mlp.predict(xvalid)
print (f1_score(yvalid, preds_valid))

0.7963386727688786


In [45]:
# prediction on test dataset
preds_test = mlp.predict(test_final)


# prepare submission dataframe
sub = pd.DataFrame({'id':test_data['id'], 'label':preds_test})

# write predictions to a CSV file
sub.to_csv(save_path + "sub_mlp_bert+nnlm.csv", index=False)
#Your score for this submission is : 0.886689220510626.

## svm

In [29]:
clf = svm.SVC(gamma='scale')

clf.fit(xtrain, ytrain)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [30]:
preds_valid = clf.predict(xvalid)
print (f1_score(yvalid, preds_valid))

0.770909090909091


In [31]:
# prediction on test dataset
preds_test = clf.predict(test_final)


# prepare submission dataframe
sub = pd.DataFrame({'id':test_data['id'], 'label':preds_test})

# write predictions to a CSV file
sub.to_csv(save_path + "sub_svm_bert+nnlm.csv", index=False)
#Your score for this submission is : 0.8654970760233918.